In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-11-21 09:49:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2023-11-21 09:49:40 (8.26 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()
print(f"Total length of the text is {len(text)}")

Total length of the text is 1115394


In [2]:
#Create the vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # Given a string return a list of intgers represeting each character
decode = lambda l: ''.join([itos[i] for i in l]) # Given a list of integers return the string representation

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(2803)
torch.cuda.is_available()

False

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
# split into training and validation test data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [101]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 192
n_head = 6
n_layer = 6
dropout = 0.2

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [102]:
class Head(nn.Module):
    """ one head of self attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)

        #compute the ateention scores ("affinites")
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        #perform the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v
        return out



In [103]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [104]:
class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

In [105]:
class Block(nn.Module):
    """ Transformer Block - Intersperses computations (Feedforward) and communication (self-attention)"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [106]:
# Implement a Bigram langauage model using pytorch
class BigramLanugageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.block = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd);
        self.lm_head = nn.Linear(n_embd, vocab_size)
    

    def forward(self, idx, targets=None):
        B,T = idx.shape
        
        tok_embd = self.token_embedding_table(idx) # (B, T, C)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_embd + pos_embd # works because of broadcasting
        x = self.block(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            # some shit about corss entropy function require the below change in shaping
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a B, T tensor, this function will generate a (B, T+1), (B, T+2) ... (B, T+max_new_token) tensor
        for _ in range(max_new_tokens):

            # crop the idx to be only upto block size
            idx_cond = idx[:, -block_size:]
            
            # get the prediction
            logits, loss = self(idx_cond)

            # focus only on the last time step
            logits = logits[:, -1, :] # becomes B, C

            # softmax to get probabilities
            probs = F.softmax(logits, dim=-1) #(B, C)

            #sample from the distrobution
            idx_next = torch.multinomial(probs, replacement=True, num_samples=1) # (B, 1)

            #append sampled index to the running sequence 
            idx = torch.cat((idx, idx_next), dim=1)
        return idx




In [107]:
model = BigramLanugageModel()
m = model.to(device)
#Generate a optimizer object
optimizer =  torch.optim.AdamW(m.parameters(), lr=1e-3)

In [108]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out


In [109]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3820, val loss 4.3752
step 500: train loss 1.7731, val loss 1.8997
step 1000: train loss 1.5584, val loss 1.7363
step 1500: train loss 1.4685, val loss 1.6576
step 2000: train loss 1.4226, val loss 1.6263
step 2500: train loss 1.3835, val loss 1.5936
step 3000: train loss 1.3531, val loss 1.5869


In [100]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


vult septing: he right,
And, of evence, of truntoon yeth were 'titly alighname thiughtern upone-Clal dead ancil,
Did veren fight of the greon a condien,
A my such orlly styw th be?
Totht sorn way, for minstly setted:
I have hoven nem lord,
Lording mady Nay, that why it; brick, nother it stes,
Is sness may aly for him ning now, on, they.

BENVOLY:
Shint,
We my sens, by it son the of anow;
Lu, ther necl, that I courturant pry of the sents,
What I looodd's yiak one burn
The sones aludy, your arm th


In [74]:
x = torch.tensor([[[1, 2, 3], [4,5, 6], [7, 8, 9]]])
print(x.shape)
print(x)
x.transpose(-2, -1)

torch.Size([1, 3, 3])
tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])


tensor([[[1, 4, 7],
         [2, 5, 8],
         [3, 6, 9]]])

In [21]:
#Self attention
torch.manual_seed(1337)
B, T, C = 1, 2, 2
x = torch.randn(B,T,C)

tril = torch.tril(torch.ones(T, T))
head_size = 1
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)  # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei = q @ k.transpose(-2, -1) # (B, T, head_size) @ (B, head_size, T) ----> (B, T, T)
print(wei[0])

wei = wei.masked_fill(tril == 0 , float('-Inf'))  
print("---------")
print(wei[0])
wei = F.softmax(wei, dim=1)
print("---------")
print(wei[0])
print(wei.shape)
v = value(x)   # (B, T, head_size)
print(v.shape)
out = wei @ v  # (B, T, T)  @ (B, T, head_size) ---> (B, T, head_size)
print(out)


tensor([[-1.3597, -0.7191],
        [-0.7499, -0.3966]], grad_fn=<SelectBackward0>)
---------
tensor([[-1.3597,    -inf],
        [-0.7499, -0.3966]], grad_fn=<SelectBackward0>)
---------
tensor([[0.3521, 0.0000],
        [0.6479, 1.0000]], grad_fn=<SelectBackward0>)
torch.Size([1, 2, 2])
torch.Size([1, 2, 1])
tensor([[[0.1720],
         [0.7268]]], grad_fn=<UnsafeViewBackward0>)


In [13]:
D = torch.tensor([-1.7629, -3.3334, -1.0226, 0.7836, -1.2566, -0.3126, 1.0876, -1.8044], dtype=float)
print(D.dim(), D.shape)
F.softmax(D)

1 torch.Size([8])


/var/folders/xz/m4ylstqx7c7bqd0wfj5n6z780000gn/T/ipykernel_11748/281031289.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(D)


tensor([0.0248, 0.0052, 0.0521, 0.3171, 0.0412, 0.1060, 0.4298, 0.0238],
       dtype=torch.float64)